## Credit Score Classification

Final project for COMP 562-001 by Jordan Bean (PID: 730151647).

In [1]:
import pandas as pd
import numpy as np

# Read data into a dataframe
data = pd.read_csv('data/train.csv')

# Display top of dataframe
data.head(10)

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
# Check data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

### (1) Data Cleaning

In [3]:
# Replace all occurrences of Not Available with numpy not a number
data = data.replace({'Not Available': np.nan})

# Iterate through the columns, remove unnecessary characters (if applicable), and convert to appropriate data type
for col in list(data.columns):
    if ('Month' == col or 'Name' in col or 'SSN' in col or 'Occupation' in col or 'Credit_Mix' in col or
        'Credit_History_Age' in col or 'ID' in col or 'Type_of_Loan' in col or 
        'Payment_of_Min_Amount' in col or 'Payment_Behaviour' in col or 
        'Credit_Score' in col):
        data[col] = data[col].astype('string')
    elif ('Age' in col or 'Num_of_Loan' in col):
        data[col] = data[col].astype('string')
        data[col] = data[col].str.replace("_", "")
        data[col] = data[col].str.replace("-", "")
        data[col] = data[col].astype('int')
        # data[col].where(data['Age'] > 99, np.nan, inplace=True)
    elif ('Annual_Income' in col or 'Outstanding_Debt' in col or 
        'Monthly_Balance' in col or 'Amount_invested_monthly' in col):
        data[col] = data[col].str.replace("_", "")
        data[col] = data[col].astype('float')

# Could not convert Num_of_Delayed_Payment or Changed_Credit_Limit

In [4]:
# Check data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  string 
 1   Customer_ID               100000 non-null  string 
 2   Month                     100000 non-null  string 
 3   Name                      90015 non-null   string 
 4   Age                       100000 non-null  int64  
 5   SSN                       100000 non-null  string 
 6   Occupation                100000 non-null  string 
 7   Annual_Income             100000 non-null  float64
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  int64  
 13  Type_of_Loan              88592 non-null   st

In [5]:
# Function to calculate missing values by column
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [6]:
missing_values_table(data)

Your selected dataframe has 28 columns.
There are 8 columns that have missing values.


,Missing Values,% of Total Values
Monthly_Inhand_Salary,15002,15.0
Type_of_Loan,11408,11.4
Name,9985,10.0
Credit_History_Age,9030,9.0
Num_of_Delayed_Payment,7002,7.0
Amount_invested_monthly,4479,4.5
Monthly_Balance,2868,2.9
Num_Credit_Inquiries,1965,2.0


In [7]:
# Get the columns with > 50% missing
missing_df = missing_values_table(data);
missing_columns = list(missing_df[missing_df['% of Total Values'] > 50].index)
print('We will remove %d columns.' % len(missing_columns))

Your selected dataframe has 28 columns.
There are 8 columns that have missing values.
We will remove 0 columns.


### (2) Exploratory Analysis